# Import libraries

In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import pandas as pd
import numpy as np
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Activation, BatchNormalization, Dropout, Input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import keras
from datetime import datetime
from sklearn import preprocessing
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import plot_model
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
import pickle
import glob

Using TensorFlow backend.


# Load processed dataset
The data we load here are already shuffled and normalized.

In [2]:
data_directory = 'dataset/3. normalized/'
x_train = pickle.load(open(data_directory + "x_train.p", "rb"))
y_train = pickle.load(open(data_directory + "y_train.p", "rb"))
x_test = pickle.load(open(data_directory + "x_test.p", "rb"))
y_test = pickle.load(open(data_directory + "y_test.p", "rb"))
print('x_train: ' + str(x_train.shape))
print('y_train: ' + str(y_train.shape))
print('x_test: ' + str(x_test.shape))
print('y_test: ' + str(y_test.shape))

x_train: (966, 26)
y_train: (966, 2)
x_test: (242, 26)
y_test: (242, 2)


# Tunable parameters

In [3]:
epochs = 1300
batch_size = 20
learning_rate = 0.0001
decay = 0.0001
dropout = 0.4

# Autoencoder #2

In [4]:
date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
root_directory = 'results/results-autoencoder-02/'
autoencoder_1_directory = 'results/results-autoencoder-01/model/'
model_directory = root_directory + "model/"
tensorboard_directory = root_directory + "tensorboard/"
autoencoder_1_path = ''
def create_model(layers):
    global date
    global autoencoder_1_path

    x = Input(shape=(x_train.shape[1],))

    y = Dense(units = layers['layer_1'], activation='relu')(x)
    y = Dropout(dropout)(y)
    
    y = Dense(units = x_train.shape[1], activation='sigmoid')(y)
    model = None
    model = Model(x, y)
    
    # Create directory
    directory = model_directory + date + '/'
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # Write model hyper-parameters
    df = pd.DataFrame({
        'Layer 1': [layers['layer_1']],
        'Dropout': [dropout],
        'Learning Rate': [learning_rate],
        'Decay': [decay],
        'Autoencoder 1': [str(autencoder_1_path)]
    })
    df.to_csv(model_directory + date + '/params.csv')
    
    # Write model summary
    file2 = open(directory + "summary.txt", "a")
    model.summary(print_fn=lambda line: file2.write(line + '\n'))
    file2.close()

    # Write model diagram
    plot_model(model, to_file=directory + 'model.png', show_shapes=True, show_layer_names=False)
    
    # Compile the model
    model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate, decay=decay), loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
    
    return model

# Prepare callbacks

In [5]:
class KerasClassifierTensorBoard(KerasClassifier):
    def fit(self, x, y, **kwargs):
        global date
        date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
        tensorboard_callback = keras.callbacks.TensorBoard(log_dir=tensorboard_directory + date + '/')
        csv_logger = keras.callbacks.CSVLogger(model_directory + date + '/epochs.csv')
        model_checkpoint = keras.callbacks.ModelCheckpoint(model_directory + date + '/weights.hdf5', save_best_only=True)
        callbacks = [tensorboard_callback, csv_logger, model_checkpoint]
        return super(KerasClassifierTensorBoard, self).fit(x, y, callbacks=callbacks, verbose=0, **kwargs)

# Get autoencoder #1 paths

In [6]:
weight_paths = glob.glob(autoencoder_1_directory + '*/weights.hdf5')

# Grid search

In [7]:
for weight_path in weight_paths:
    autoencoder_1 = load_model(weight_path)
    autoencoder_1_output = autoencoder_1.predict(x_train)
    autoencoder_1_output_test = autoencoder_1.predict(x_test)
    autencoder_1_path = weight_path
    
    model = None
    model = KerasClassifierTensorBoard(build_fn=create_model, epochs=epochs, batch_size=batch_size)

    layers = [
        {'layer_1': 10},
        {'layer_1': 8},
        {'layer_1': 6},
        {'layer_1': 4}
    ]
    param_grid = dict(
        layers=layers
    )

    grid = GridSearchCV(estimator = model, param_grid = param_grid, error_score=0, verbose=2, cv=[(slice(None), slice(None))], n_jobs=1, fit_params=dict(validation_data=(autoencoder_1_output_test, autoencoder_1_output_test)))
    grid_result = grid.fit(autoencoder_1_output, autoencoder_1_output)

from search: results/results-autoencoder-01/model\2018-03-04-20-57-52\weights.hdf5
Fitting 1 folds for each of 4 candidates, totalling 4 fits
[CV] layers={'layer_1': 10} ..........................................
from model: results/results-autoencoder-01/model\2018-03-04-20-57-52\weights.hdf5
 20/966 [..............................] - ETA: 0s[CV] ........................... layers={'layer_1': 10}, total= 3.8min
[CV] layers={'layer_1': 8} ...........................................
from model: results/results-autoencoder-01/model\2018-03-04-20-57-52\weights.hdf5


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.8min remaining:    0.0s


 20/966 [..............................] - ETA: 0s[CV] ............................ layers={'layer_1': 8}, total= 3.9min
[CV] layers={'layer_1': 6} ...........................................
from model: results/results-autoencoder-01/model\2018-03-04-20-57-52\weights.hdf5
 20/966 [..............................] - ETA: 0s[CV] ............................ layers={'layer_1': 6}, total= 3.9min
[CV] layers={'layer_1': 4} ...........................................
from model: results/results-autoencoder-01/model\2018-03-04-20-57-52\weights.hdf5
 20/966 [..............................] - ETA: 0s[CV] ............................ layers={'layer_1': 4}, total= 4.0min
from model: results/results-autoencoder-01/model\2018-03-04-20-57-52\weights.hdf5


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 15.6min finished


from search: results/results-autoencoder-01/model\2018-03-04-21-04-24\weights.hdf5
Fitting 1 folds for each of 4 candidates, totalling 4 fits
[CV] layers={'layer_1': 10} ..........................................
from model: results/results-autoencoder-01/model\2018-03-04-21-04-24\weights.hdf5
800/966 [=======================>......] - ETA: 0s[CV] ........................... layers={'layer_1': 10}, total= 4.2min
[CV] layers={'layer_1': 8} ...........................................
from model: results/results-autoencoder-01/model\2018-03-04-21-04-24\weights.hdf5


...

[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 21.6min finished


from search: results/results-autoencoder-01/model\2018-03-04-22-26-57\weights.hdf5
Fitting 1 folds for each of 4 candidates, totalling 4 fits
[CV] layers={'layer_1': 10} ..........................................
from model: results/results-autoencoder-01/model\2018-03-04-22-26-57\weights.hdf5
640/966 [==================>...........] - ETA: 0s[CV] ........................... layers={'layer_1': 10}, total= 5.4min
[CV] layers={'layer_1': 8} ...........................................
from model: results/results-autoencoder-01/model\2018-03-04-22-26-57\weights.hdf5


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  5.4min remaining:    0.0s


780/966 [=======================>......] - ETA: 0s[CV] ............................ layers={'layer_1': 8}, total= 5.4min
[CV] layers={'layer_1': 6} ...........................................
from model: results/results-autoencoder-01/model\2018-03-04-22-26-57\weights.hdf5
720/966 [=====================>........] - ETA: 0s[CV] ............................ layers={'layer_1': 6}, total= 5.7min
[CV] layers={'layer_1': 4} ...........................................
from model: results/results-autoencoder-01/model\2018-03-04-22-26-57\weights.hdf5
560/966 [================>.............] - ETA: 0s[CV] ............................ layers={'layer_1': 4}, total= 5.7min
from model: results/results-autoencoder-01/model\2018-03-04-22-26-57\weights.hdf5


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 22.2min finished
